<a href="https://colab.research.google.com/github/mmartigani/RoadMapIA/blob/main/CapasPersonalizadas_con_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras import datasets

In [2]:
boston_housing = datasets.boston_housing

In [3]:
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

57026/57026 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
import pandas as pd
import numpy as np

features = ["CRIM", "ZN", "INDUS", "CHAS", "NOX", "RM", "AGE", "DIS", "RAD", "TAX", "PTRATIO", "B", "LSTAT", "MEDV"]

df_train = pd.DataFrame(np.column_stack([X_train, y_train]), columns=features)
df_train.head(10)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,1.23247,0.0,8.14,0.0,0.538,6.142,91.7,3.9769,4.0,307.0,21.0,396.90,18.72,15.2
1,0.02177,82.5,2.03,0.0,0.415,7.610,15.7,6.2700,2.0,348.0,14.7,395.38,3.11,42.3
2,4.89822,0.0,18.10,0.0,0.631,4.970,100.0,1.3325,24.0,666.0,20.2,375.52,3.26,50.0
3,0.03961,0.0,5.19,0.0,0.515,6.037,34.5,5.9853,5.0,224.0,20.2,396.90,8.01,21.1
4,3.69311,0.0,18.10,0.0,0.713,6.376,88.4,2.5671,24.0,666.0,20.2,391.43,14.65,17.7
5,0.28392,0.0,7.38,0.0,0.493,5.708,74.3,4.7211,5.0,287.0,19.6,391.13,11.74,18.5
6,9.18702,0.0,18.10,0.0,0.700,5.536,100.0,1.5804,24.0,666.0,20.2,396.90,23.60,11.3
7,4.09740,0.0,19.58,0.0,0.871,5.468,100.0,1.4118,5.0,403.0,14.7,396.90,26.42,15.6
8,2.15505,0.0,19.58,0.0,0.871,5.628,100.0,1.5166,5.0,403.0,14.7,169.27,16.65,15.6
9,1.62864,0.0,21.89,0.0,0.624,5.019,100.0,1.4394,4.0,437.0,21.2,396.90,34.41,14.4


In [5]:
from sklearn.model_selection import train_test_split
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5)

In [6]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()

X_train_prep = scaler.fit_transform(X_train)
X_val_prep = scaler.transform(X_val)
X_test_prep = scaler.transform(X_test)

In [7]:
from tensorflow import keras
import tensorflow as tf

# Capa personalizada que eleva al cuadrado los valores de entrada
square_layer = keras.layers.Lambda(lambda x: tf.square(x))

In [8]:
from tensorflow.keras import models
from tensorflow.keras import layers

In [9]:
network = models.Sequential()
network.add(layers.Dense(30, activation='relu', input_shape=X_train.shape[1:]))
network.add(layers.Dense(10, activation='relu'))
network.add(square_layer)
network.add(layers.Dense(1))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
network.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 30)             │           420 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │           310 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lambda (Lambda)                 │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 741 (2.89 KB)

 Trainable params: 741 (2.89 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
network.compile(
    loss='mean_squared_error',
    optimizer='adam',
    metrics=['mae'])

In [12]:
history = network.fit(X_train_prep,
                      y_train,
                      epochs=50,
                      validation_data=(X_val_prep, y_val))

Epoch 1/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 2s 21ms/step - loss: 591.7622 - mae: 22.4205 - val_loss: 514.6812 - val_mae: 20.9646
Epoch 2/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 560.9626 - mae: 21.6028 - val_loss: 490.6002 - val_mae: 20.3202
Epoch 3/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 545.6628 - mae: 20.9174 - val_loss: 462.8155 - val_mae: 19.8635
Epoch 4/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 489.2609 - mae: 20.1658 - val_loss: 425.8293 - val_mae: 18.9116
Epoch 5/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 484.7924 - mae: 19.8089 - val_loss: 380.6880 - val_mae: 17.7234
Epoch 6/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 407.3793 - mae: 18.0288 - val_loss: 348.5733 - val_mae: 17.0278
Epoch 7/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 396.9775 - mae: 17.6507 - val_loss: 328.2504 - val_mae: 16.4789
Epoch 8/50
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 358.5070 - mae: 16.7250 - val_loss: 310.6595 - val_mae: 15.9768
Epoch 9/50
13/13 ━━━━━

In [13]:
network.predict(X_test_prep)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step 


array([[14.80995 ],
       [25.973734],
       [35.12072 ],
       [30.917253],
       [ 8.050225],
       [20.856014],
       [23.878998],
       [23.36528 ],
       [20.93928 ],
       [38.883865],
       [16.674623],
       [17.748241],
       [10.843628],
       [24.12617 ],
       [24.99444 ],
       [26.799797],
       [38.59275 ],
       [21.195917],
       [17.857033],
       [46.771038],
       [20.25043 ],
       [26.838627],
       [23.609005],
       [21.942791],
       [12.101771],
       [21.175913],
       [28.505426],
       [16.058453],
       [20.64558 ],
       [20.36981 ],
       [23.21369 ],
       [39.081818],
       [20.751907],
       [23.828999],
       [25.162231],
       [15.252954],
       [48.40452 ],
       [32.301903],
       [22.23231 ],
       [25.74964 ],
       [15.729224],
       [21.666801],
       [18.636997],
       [14.726369],
       [20.469492],
       [29.651928],
       [33.7015  ],
       [24.898949],
       [23.994383],
       [16.478106],
